# 7. Neural Machine Translation and Models with Attention

* http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture9.pdf
* http://web.stanford.edu/class/cs224n/lectures/cs224n-2017-lecture10.pdf
* https://arxiv.org/pdf/1409.0473.pdf

In [33]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import nltk
import random
import numpy as np
from collections import Counter, OrderedDict
import nltk
from copy import deepcopy
import os
import re
flatten = lambda l: [item for sublist in l for item in sublist]

In [2]:
USE_CUDA = torch.cuda.is_available()

FloatTensor = torch.cuda.FloatTensor if USE_CUDA else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if USE_CUDA else torch.LongTensor
ByteTensor = torch.cuda.ByteTensor if USE_CUDA else torch.ByteTensor

In [264]:
def getBatch_FromBucket(batch_size,buckets):
    i=0
    bucket_mask =[False for _ in range(len(buckets))]
    indices = [[0,batch_size] for _ in range(len(buckets))]
    is_done=False
    while is_done==False:
        batch = buckets[i][indices[i][0]:indices[i][1]]
        temp = indices[i][1]
        indices[i][1]= indices[i][1]+batch_size
        indices[i][0] = temp
        
        i = (i+1)%len(buckets)
        while bucket_mask[i]:
            i = (i+1)%len(buckets)
        
        if indices[i][1]>len(buckets[i]):
            bucket_mask[i]= True
            if bucket_mask.count(True)==len(buckets):
                is_done=True
            else:
                while bucket_mask[i]:
                    i = (i+1)%len(buckets)
        yield batch


In [4]:
def pad_to_batch(batch,x_to_ix,y_to_ix):
    x,y = list(zip(*batch))
    max_x = max([s.size(1) for s in x])
    max_y = max([s.size(1) for s in y])
    x_p,y_p=[],[]
    for i in range(len(batch)):
        if x[i].size(1)<max_x:
            x_p.append(torch.cat([x[i],Variable(LongTensor([x_to_ix['<PAD>']]*(max_x-x[i].size(1)))).view(1,-1)],1))
        else:
            x_p.append(x[i])
        if y[i].size(1)<max_y:
            y_p.append(torch.cat([y[i],Variable(LongTensor([y_to_ix['<PAD>']]*(max_y-y[i].size(1)))).view(1,-1)],1))
        else:
            y_p.append(y[i])
    return torch.cat(x_p),torch.cat(y_p)

In [25]:
def prepare_sequence(seq, to_index):
    idxs = list(map(lambda w: to_index[w] if w in to_index.keys() else to_index["<UNK>"], seq))
    return Variable(LongTensor(idxs))

### Data load and Preprocessing 

* https://github.com/chiphuyen/stanford-tensorflow-tutorials/tree/master/assignments/chatbot

In [31]:
def basic_tokenizer(line, normalize_digits=True):
    """ A basic tokenizer to tokenize text into tokens.
    Feel free to change this to suit your need. """
    line = re.sub('<u>', '', line)
    line = re.sub('</u>', '', line)
    line = re.sub('\[', '', line)
    line = re.sub('\]', '', line)
    words = []
    _WORD_SPLIT = re.compile("([.,!?\"'-<>:;)(])")
    _DIGIT_RE = re.compile("\d")
    for fragment in line.strip().lower().split():
        for token in re.split(_WORD_SPLIT, fragment):
            if not token:
                continue
            if normalize_digits:
                token = re.sub(_DIGIT_RE, '#', token)
            words.append(token)
    return words

In [27]:
source_corpus = open('../dataset/correll_movie/train.enc','r',encoding='latin-1').readlines()
target_corpus = open('../dataset/correll_movie/train.dec','r',encoding='latin-1').readlines()

In [28]:
print(len(source_corpus)==len(target_corpus))
print(len(source_corpus))

True
196616


In [34]:
%%time
X_r,y_r=[],[] # raw

for parallel in zip(source_corpus,target_corpus):
    if parallel[1]=='\n':continue
    so,ta = parallel[0][:-1], parallel[1][:-1]
    if so.strip()=="" or ta.strip()=="": continue
    X_r.append(basic_tokenizer(so))
    y_r.append(basic_tokenizer(ta))

CPU times: user 11.7 s, sys: 96 ms, total: 11.8 s
Wall time: 11.8 s


### Build Vocab

In [35]:
source_vocab = list(set(flatten(X_r)))
target_vocab = list(set(flatten(y_r)))
print(len(source_vocab),len(target_vocab))

40625 40915


In [70]:
VOCAB_LIMIT = 20000

In [71]:
source_vocab_count = Counter(flatten(X_r))
source_vocab, _ = list(zip(*source_vocab_count.most_common()[:VOCAB_LIMIT]))

target_vocab_count = Counter(flatten(y_r))
target_vocab, _  = list(zip(*target_vocab_count.most_common()[:VOCAB_LIMIT]))
print(len(source_vocab),len(target_vocab))

20000 20000


In [ ]:
# source_corpus = open('./dataset/iwslt15/train.en','r',encoding='utf-8').readlines()
# target_corpus = open('./dataset/iwslt15/train.vi','r',encoding='utf-8').readlines()

In [9]:
# source_vocab = open('./dataset/iwslt15/vocab.en','r',encoding='utf-8').readlines()
# source_vocab = [s[:-1] for s in source_vocab]
# target_vocab = open('./dataset/iwslt15/vocab.vi','r',encoding='utf-8').readlines()
# target_vocab = [t[:-1] for t in target_vocab]
# print(len(source_vocab),len(target_vocab))

17191 7709


In [72]:
source2index = {'<PAD>':0,'<UNK>':1,'<s>':2,'</s>':3}
for vo in source_vocab:
    source2index[vo]=len(source2index)
index2source = {v:k for k,v in source2index.items()}

target2index = {'<PAD>':0,'<UNK>':1,'<s>':2,'</s>':3}
for vo in target_vocab:
    target2index[vo]=len(target2index)
index2target = {v:k for k,v in target2index.items()}

In [37]:
so_length_dist = Counter([len(x) for x in X_r])
ta_length_dist = Counter([len(y) for y in y_r])

In [47]:
print(np.mean(flatten([[s[0]]*s[1] for s in so_length_dist.most_common()])))
print(np.mean(flatten([[t[0]]*t[1] for t in ta_length_dist.most_common()])))

14.4687952077
15.0052109864


In [103]:
MAX_LENGTH=20 # because of unsufficient memory

In [104]:
%%time
X_p,y_p=[],[]

for so,ta in zip(X_r,y_r):
    if len(so)>MAX_LENGTH or len(ta)>MAX_LENGTH: continue
    X_p.append(prepare_sequence(['<s>']+so+['</s>'],source2index).view(1,-1))
    y_p.append(prepare_sequence(ta+['</s>'],target2index).view(1,-1))
    
train_data = list(zip(X_p,y_p))

CPU times: user 6.32 s, sys: 60 ms, total: 6.38 s
Wall time: 6.38 s


### Bucket 

In [109]:
BUCKET_CONFIG =[(8, 10), (12, 14), (16, 20)]

In [146]:
buckets = [[],[],[]]

In [147]:
for so,ta in train_data:
    so_length = so.size(1)
    ta_length = ta.size(1)
    for i in range(len(BUCKET_CONFIG)):
        if BUCKET_CONFIG[i][0] >= so_length and BUCKET_CONFIG[i][1] >= ta_length:
            
            buckets[i].append((so,ta))
            break

In [148]:
for b in buckets:
    print(len(b))

29793
36731
34628


### Modeling 

In [98]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_size,hidden_size, n_layers=1,bidirec=False):
        super(Encoder, self).__init__()
        
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        if bidirec:
            self.n_direction = 2 
            self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True,bidirectional=True)
        else:
            self.n_direction = 1
            self.gru = nn.GRU(embedding_size, hidden_size, n_layers, batch_first=True)
    
    def init_hidden(self,inputs):
        hidden = Variable(torch.zeros(self.n_layers*self.n_direction,inputs.size(0),self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden
    
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_uniform(self.embedding.weight)
#         self.gru.weight_hh_l0 = nn.init.xavier_uniform(self.gru.weight_hh_l0)
#         self.gru.weight_ih_l0 = nn.init.xavier_uniform(self.gru.weight_ih_l0)
    
    def forward(self, inputs, input_masking=None):
        """
        inputs : B,T (LongTensor)
        input_masking : B,T (ByteTensor) if you don't use zero-padding, leave it at that
        """
        hidden = self.init_hidden(inputs)
        
        embedded = self.embedding(inputs)
        output, hidden = self.gru(embedded, hidden)

        real_context=[]
        
        if type(input_masking)==torch.autograd.variable.Variable:
            for i,o in enumerate(output): # B,T,D
                real_length = input_masking[i].data.tolist().count(0) # real length
                real_context.append(o[real_length-1])
            hidden = torch.cat(real_context).view(inputs.size(0),-1).unsqueeze(1)
        else:
            hidden = torch.cat(hidden,1).unsqueeze(1)
        
        return output, hidden

In [99]:
class Decoder(nn.Module):
    def __init__(self, input_size, embedding_size, hidden_size, n_layers=1,dropout_p=0.3):
        super(Decoder, self).__init__()
        
        self.hidden_size = hidden_size
        self.n_layers = n_layers
        
        # Define the layers
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.dropout = nn.Dropout(dropout_p)
        
        self.gru = nn.GRU(embedding_size+hidden_size, hidden_size, n_layers,batch_first=True)
        self.linear = nn.Linear(hidden_size*2, input_size)
        self.attn = nn.Linear(self.hidden_size,self.hidden_size) # Attention
        
    def init_hidden(self,inputs):
        hidden = Variable(torch.zeros(self.n_layers,inputs.size(0),self.hidden_size))
        return hidden.cuda() if USE_CUDA else hidden
    
    
    def init_weight(self):
        self.embedding.weight = nn.init.xavier_uniform(self.embedding.weight)
#         self.gru.weight_hh_l0 = nn.init.xavier_uniform(self.gru.weight_hh_l0)
#         self.gru.weight_ih_l0 = nn.init.xavier_uniform(self.gru.weight_ih_l0)
        self.linear.weight = nn.init.xavier_uniform(self.linear.weight)
    
    
    def Attention(self, hidden, encoder_outputs, encoder_maskings):
        """
        hidden : 1,B,D
        encoder_outputs : B,T,D
        encoder_maskings : B,T # ByteTensor
        """
        
        hidden = hidden.squeeze(0).unsqueeze(2)  # 히든 : (1,B,D) -> (B,D,1)
        
        batch_size = encoder_outputs.size(0) # B
        max_len = encoder_outputs.size(1) # T
        energies = self.attn(encoder_outputs.contiguous().view(batch_size*max_len,-1)) # B*T,D -> B*T,D
        energies = energies.view(batch_size,max_len,-1) # B,T,D
        attn_energies = energies.bmm(hidden).transpose(1,2).squeeze(1) # B,T,D * B,D,1 --> B,T
        
        if type(encoder_maskings)==torch.autograd.variable.Variable:
            attn_energies = attn_energies.masked_fill(encoder_maskings,-1e12) # PAD masking
        
        alpha = F.softmax(attn_energies) # B,T
        alpha = alpha.unsqueeze(1) # B,1,T
        context = alpha.bmm(encoder_outputs) # B,1,T * B,T,D => B,1,D
        
        return context # B,1,D
    
    
    def forward(self,inputs,context,max_length,encoder_outputs,encoder_maskings=None,training=False):
        """
        inputs : B,1 (LongTensor, START SYMBOL)
        context : B,1,D (FloatTensor, Last encoder hidden state)
        encoder_outputs : B,T,D
        encoder_maskings : B,T # ByteTensor
        max_length : int, max length to decode
        training : bool, this is because adapt dropout only training step.
        """
        # Get the embedding of the current input word
        embedded = self.embedding(inputs)
        hidden = self.init_hidden(inputs)
        if training:
            embedded = self.dropout(embedded)
        
        decode=[]
        # Apply GRU to the output so far
        for i in range(max_length):

            _, hidden = self.gru(torch.cat((embedded,context),2), hidden) # h_t = f(h_{t-1},y_{t-1},c)
            concated = torch.cat((hidden,context.transpose(0,1)),2) # y_t = g(h_t,y_{t-1},c)
            score = self.linear(concated.squeeze(0))
            softmaxed = F.log_softmax(score)
            decode.append(softmaxed)
            decoded = softmaxed.max(1)[1]
            embedded = self.embedding(decoded).unsqueeze(1) # y_{t-1}
            if training:
                embedded = self.dropout(embedded)
            
            # compute next context vector using attention
            context = self.Attention(hidden, encoder_outputs,encoder_maskings) 
        #  column-wise concat, reshape!!
        scores = torch.cat(decode,1)
        return scores.view(inputs.size(0)*max_length,-1)

### Train 

It takes for a while...

In [265]:
STEP=20
BATCH_SIZE = 64
EMBEDDING_SIZE = 100
HIDDEN_SIZE = 128
LR = 0.001

In [266]:
encoder = Encoder(len(source2index),EMBEDDING_SIZE,HIDDEN_SIZE,3,True)
decoder = Decoder(len(target2index),EMBEDDING_SIZE,HIDDEN_SIZE*2)
encoder.init_weight()
decoder.init_weight()

if USE_CUDA:
    encoder = encoder.cuda()
    decoder = decoder.cuda()

loss_function = nn.CrossEntropyLoss(ignore_index=0)
enc_optimizer = optim.Adam(encoder.parameters(),lr=LR)
dec_optimizer = optim.Adam(decoder.parameters(),lr=LR)

In [ ]:
# encoder.load_state_dict(torch.load('../encoder.pkl'))
# decoder.load_state_dict(torch.load('../decoder.pkl'))

In [267]:
for step in range(STEP):
    losses=[]
    for i,batch in enumerate(getBatch_FromBucket(BATCH_SIZE,buckets)):
        inputs,targets = pad_to_batch(batch,source2index,target2index)
        
        input_mask = torch.cat([Variable(ByteTensor(tuple(map(lambda s: s ==0, t.data))),volatile=False) for t in inputs]).view(inputs.size(0),-1)
        start_decode = Variable(LongTensor([[target2index['<s>']]*targets.size(0)])).transpose(0,1)
        
        encoder.zero_grad()
        decoder.zero_grad()
        output, hidden_c = encoder(inputs,input_mask)
        
        preds = decoder(start_decode,hidden_c,targets.size(1),output,input_mask,True)
                                
        loss = loss_function(preds,targets.view(-1))
        losses.append(loss.data.cpu().numpy()[0] if USE_CUDA else loss.data.numpy()[0] )
        loss.backward()
        torch.nn.utils.clip_grad_norm(encoder.parameters(), 0.5) # gradient clipping
        torch.nn.utils.clip_grad_norm(decoder.parameters(), 0.5) # gradient clipping
        enc_optimizer.step()
        dec_optimizer.step()

    
        if i % 500==0:
            print("[%d/%d] [%d/%d] mean_loss : %0.2f" %(step,STEP,i,len(train_data)//BATCH_SIZE,np.mean(losses)))
            torch.save(decoder.state_dict(),'./model/decoder.pkl')
            torch.save(encoder.state_dict(),'./model/encoder.pkl')
            losses=[]
    
#     if (step+1) % 10 == 0:
#         LR = LR/2
#         enc_optimizer = optim.Adam(encoder.parameters(),lr=LR)
#         dec_optimizer = optim.Adam(decoder.parameters(),lr=LR)

[0/20] [0/1915] mean_loss : 9.91
[0/20] [500/1915] mean_loss : 5.24
[0/20] [1000/1915] mean_loss : 5.01
[0/20] [1500/1915] mean_loss : 5.02
[1/20] [0/1915] mean_loss : 5.28
[1/20] [500/1915] mean_loss : 4.91
[1/20] [1000/1915] mean_loss : 4.88


KeyboardInterrupt: 

### Test

In [68]:
test = random.choice(test_data)
input_ = prepare_sequence(tknz.tokenize(test[0].lower()),word2index).view(1,-1)
input_mask = Variable(ByteTensor(tuple(map(lambda s: s ==0, input_[0].data)))).view(1,-1)
start_decode = Variable(LongTensor([[ta2index['<s>']]*1])).transpose(0,1)
o, hidden_c = encoder(input_,input_mask)
pred = decoder(start_decode,hidden_c,10)
pred = pred.max(1)[1].data.cpu().tolist() if USE_CUDA else pred.max(1)[1].data.tolist()

print(' '.join(test[0]))
print(' '.join(test[1]))
print(' '.join([index2word[t] for t in pred])) 

hi
USER : hi
BOT : i can t know what i don talking about .
do you know kimchi?
USER : do you know kimchi?
BOT : hell ! ! ! the the kimchi !! victoria !
do you know Jisung park?
USER : do you know Jisung park?
BOT : yes . i idea to idea ? about idea ?
hmm
USER : hmm
BOT : it is a matter . i just 't saying .
what is your name
USER : what is your name
BOT : what me t understand :( . i me to discuss
fuck you
USER : fuck you
BOT : yes are are name ? are are are about discuss
hell
USER : hell
BOT : second , go thermodynamics ! the vain , of .
you stupid
USER : you stupid
BOT : i don t know what . i would like .
have a good night
USER : have a good night
BOT : how about we to something to both . about about
fuck..
USER : fuck..
BOT : yeah . i about . your . about . .
are you chatbot?
USER : are you chatbot?
BOT : no ,,!! i 'm... . they by students . i
mm
USER : mm
BOT : you you make the first to your . the iron
bye
USER : bye
BOT : here , ! the ! , . , . ,


because of noisy data, It is hard to converge

# TODO 

* newstest data로 BLEU 측정
* Beam Search
* Sampled Softmax

In [22]:
test_en = open('../dataset/iwslt15/tst2012.en','r',encoding='utf-8').readlines()
test_vi = open('../dataset/iwslt15/tst2012.vi','r',encoding='utf-8').readlines()

In [23]:
len(test_en)

1553

In [24]:
len(test_vi)

1553